# Setup for running customization notebooks both for fine-tuning and continued pre-training using Amazon Bedrock

In this notebook, we will create set of roles and an s3 bucket which will be used for other notebooks in this module. 

## Setup
Install and import all the needed libraries and dependencies to complete this notebook.

<div class="alert alert-block alert-warning">
<b>Warning:</b> Please ignore error messages related to pip's dependency resolver.
</div>

In [1]:
!pip install --upgrade pip
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"
!pip install pandas
!pip install datasets

  Using cached jmespath-1.0.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached s3transfer-0.10.2-py3-none-any.whl.metadata (1.7 kB)
  Using cached docutils-0.16-py2.py3-none-any.whl.metadata (2.7 kB)
  Using cached PyYAML-6.0.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached colorama-0.4.6-py2.py3-none-any.whl.metadata (17 kB)
  Using cached rsa-4.7.2-py3-none-any.whl.metadata (3.6 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached urllib3-2.2.2-py3-none-any.whl.metadata (6.4 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl.metadata (1.8 kB)
  Using cached pyasn1-0.6.0-py2.py3-none-any.whl.metadata (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 22.6 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 25.6 MB/s eta 0:00:00 MB/s eta 0:00:0101
Using cached colorama-0.4.6-py2.py3-none-any.whl (25 kB)
Using cached docutils-0.16-py2.py3-none-any.whl (548 kB)
Using cached jmes

In [2]:
# restart kernel for packages to take effect
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [3]:
import pandas as pd
from datasets import Dataset

# Load the CSV file
csv_file_path = "sourcestraining.csv"  # Ensure this path points to your CSV file
df = pd.read_csv(csv_file_path, delimiter=';')  # Assuming the file is semicolon-separated

# Rename columns to match input/output for translation
df.columns = ['translation_input', 'translation_target']

# Convert the DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Split the dataset into training, validation, and test sets
train_test_split = dataset.train_test_split(test_size=0.2)
train_val_split = train_test_split['train'].train_test_split(test_size=0.1)

train_dataset = train_val_split['train']
validation_dataset = train_val_split['test']
test_dataset = train_test_split['test']

# Display the size of each split
print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(validation_dataset)}")
print(f"Test set size: {len(test_dataset)}")

/Users/xiaweiyi/bedrockdemogit/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FileNotFoundError: [Errno 2] No such file or directory: 'sourcestraining.csv'

In [ ]:
import os

# Define dataset folder and file names
dataset_folder = "fine-tuning-datasets"
train_file_name = "train.csv"
validation_file_name = "validation.csv"
test_file_name = "test.csv"

# Create the directory if it doesn't exist
os.makedirs(dataset_folder, exist_ok=True)

# Save the datasets to CSV files
train_dataset.to_pandas().to_csv(os.path.join(dataset_folder, train_file_name), index=False)
validation_dataset.to_pandas().to_csv(os.path.join(dataset_folder, validation_file_name), index=False)
test_dataset.to_pandas().to_csv(os.path.join(dataset_folder, test_file_name), index=False)

# Get the absolute path of the dataset folder
abs_path = os.path.abspath(dataset_folder)
print(f"Datasets saved to {abs_path}")

## Storing variables to be used in other notebooks if needed. 

> Please make sure to use the same kernel as used for 00_setup.ipynb for other notebooks on fine-tuning and continued pre-training. 

In [ ]:
#%store role_arn
#%store bucket_name
#%store role_name
#%store policy_arn
#%store s3_train_uri
#%store s3_validation_uri
#%store s3_test_uri